In [97]:
# Douwe: I see alot of unused imports, removing these might be clearner :)
import time
import torch

import monai
from monai.config import print_config
from monai.data import Dataset, DataLoader, CacheDataset, PersistentDataset, SmartCacheDataset
from monai.apps import DecathlonDataset
from monai.transforms import (
    MapTransform,
)

print_config()

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    AsChannelLastd,
    SqueezeDimd,
    AddChanneld,
)
#from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import pydicom

print_config()

from natsort import natsorted

MONAI version: 1.2.0
Numpy version: 1.24.4
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: /home/dspaanderman/Documents/GitProjects/HabbieDicomSort/venv/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
Pillow version: 10.0.0
Tensorboard version: 2.14.0
gdown version: 4.7.1
TorchVision version: 0.15.2+cu117
tqdm version: 4.66.1
lmdb version: 1.4.1
psutil version: 5.9.5
pandas version: 2.0.3
einops version: 0.6.1
transformers version: 4.21.3
mlflow version: 2.6.0
pynrrd version: 1.0.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

MONAI version: 1.2.0
Numpy version: 1.24.4
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = Fa

In [109]:
# Parent directory where "train" and "test" folders are located
#parent_directory = '//mnt/c/Users/ahmad/OneDrive - Delft University of Technology/studi/TUD/Internship/Clean_data/'

# DOUWE: using this directory for testing
parent_directory = 'data/'

# Function to extract label from the folder path
def extract_label(folder_path):
    # Split the folder path and take the last two elements as the label
    components = folder_path.split(os.path.sep)
    label = os.path.join(components[-1])
    return label

# Create a list of dictionaries with keys "path" and "label"
train_list = []
test_list = []

# Douwe: note you can also do the following (just to show you what you can do with python)
train_list, test_list = [], []

# Douwe: Some notes here:
# 1) each slice is now an entry in the dictionary, while we want each image to be an entry:
#   - Now this is the structure: [{'path': 'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00011.dcm', 'label': 'T1'}, {'path': 'data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00019.dcm', 'label': 'T1'}]
#   - We want this structure: [{'path': [data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00011.dcm, data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00019.dcm], 'label': 'T1'}]
# 2) you know check for each slice the label, however you can also check just for 1 of the slices (as it should be the same)
# 3) Check where you do if, else statements, now the "Train Sample" / "Test Sample" shares alot of code
# 4) I would set the key for the image to "image" instead of "path"

# Iterate through "train" and "test" folders
for split in ["Train Sample", "Test Sample"]:
    split_folder = os.path.join(parent_directory, split)
    # Iterate through DICOM folders within the split folder
    if split == "Train Sample":
        for root, _, files in os.walk(split_folder):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_file_path = os.path.join(root, file)
                    label = extract_label(root)
                    train_list.append({"path": dicom_file_path, "label": label})
    else:
        for root, _, files in os.walk(split_folder):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_file_path = os.path.join(root, file)
                    label = extract_label(root)
                    test_list.append({"path": dicom_file_path, "label": label})

# Douwe: This is how I would have done the for loop above:
train_list, test_list = [], []
for split in ["Train Sample", "Test Sample"]:
    split_folder = os.path.join(parent_directory, split)
    for root, _, files in os.walk(split_folder):
        if files:
            # This will create the following list:
            # [data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00011.dcm, data/Train Sample/LipoRadiomics-028/LipoRadiomics-028/T1/00019.dcm]
            dicom_files_path = natsorted([os.path.join(root, file) for file in files if file.lower().endswith('.dcm')])
            # Now instead of finding the label for all these files, lets do it just for the first index, Note I am not using the extract_label function you made
            label = os.path.basename(os.path.normpath(root))
            if split == "Train Sample":
                train_list.append({"image": dicom_files_path, "label": label})
            elif split == "Test Sample":
                test_list.append({"image": dicom_files_path, "label": label})

In [110]:
# Douwe: nice that you made a CustomDataset class! Some pointers
# 1) You don't have to load the pixel_array data -> this is what the LoadImaged transform is for
# 2) As said above, now every entry is a dicom file instead of a list of dicoms (also see above point 1)
# 3) The dicom tags like patient ID, and Magnetic Field strength should be converted to strings

# Define a custom Dataset class to load DICOM files and extract data
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        super().__init__(data, transform)

    def __getitem__(self, index):
        dicom_file_path = self.data[index]["path"]
        label = self.data[index]["label"]

        # Load the DICOM file and extract information
        dicom_data = pydicom.dcmread(dicom_file_path)
        
        #take image data
        pixel_data = dicom_data.pixel_array
        # DICOM header attributes
        patient_ID = dicom_data.PatientName
        if hasattr(dicom_data, 'RepetitionTime'):
            repetition_time = int(float(dicom_data.RepetitionTime))
            #print(f"Repetition Time (TR): {repetition_time}")
            
        if hasattr(dicom_data, 'EchoTime'):
            echo_time = int(float(dicom_data.EchoTime))
            #print(f"Echo Time (TE): {echo_time}")
        
        if hasattr(dicom_data, 'MagneticFieldStrength'):
            magnetic_strength = dicom_data.MagneticFieldStrength
            #print(f"Magnetic Field Strength: {round(magnetic_strength, 1)}")
        return {"image": pixel_data, "label": label, "patient_name": patient_ID, "metadata": {"repetition_time": repetition_time, "echo_time": echo_time, "magnetic_strength": magnetic_strength}}

# Douwe: This is how I would have done it
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        super().__init__(data, transform)

    def __getitem__(self, index):
        dicom_files_path = self.data[index]["image"]
        label = self.data[index]["label"]

        # Extract dicom info from first entry
        dicom_data = pydicom.dcmread(dicom_files_path[0])
        
        # DICOM header attributes
        patient_ID = str(dicom_data.PatientName)
        if hasattr(dicom_data, 'RepetitionTime'):
            repetition_time = int(float(dicom_data.RepetitionTime))
            
        if hasattr(dicom_data, 'EchoTime'):
            echo_time = int(float(dicom_data.EchoTime))
        
        if hasattr(dicom_data, 'MagneticFieldStrength'):
            magnetic_strength = str(dicom_data.MagneticFieldStrength)

        return {"image": dicom_files_path, "label": label, "patient_name": patient_ID, "metadata": {"repetition_time": repetition_time, "echo_time": echo_time, "magnetic_strength": magnetic_strength, "number slices": len(dicom_files_path)}}

# Create a MONAI dataset
train = CustomDataset(data=train_list)
test = CustomDataset(data=test_list)


In [111]:
train_dataset = monai.data.Dataset(train, transform=None)

print(f"Length of dataset is {len(train_dataset)}")

test_dataset = monai.data.Dataset(test, transform=None)

print(f"Length of dataset is {len(test_dataset)}")


Length of dataset is 4
Length of dataset is 3


In [112]:
test_dataset[0]

{'image': ['data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-9-sks4dj.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-16-sktbnn.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-2-sks46c.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-11-sktbii.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-12-sktbjj.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-18-sktbpp.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336551742176661-3-17-sktboo.dcm',
  'data/Test Sample/DESM-109/DESM-109_MR1/T1/2.16.840.1.113669.632.21.2738456351.2738456351.28336

In [115]:
# Douwe: Some general notes
# 1) You do alot of transforms on the label, this is probably because you look into segmentation examples of MONAI. Note that the "label" is just a string, e.g. T1, T2, etc.
#    Therefore why would you do any transforms on this label, like LoadImaged?
# 2) Previously it did not work because you already loaded the image, e.g. pixel_data = dicom_data.pixel_array in the CustomDataset. Now it should work

train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="image",
            image_threshold=0,
        ),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    ]
)

train_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image"], source_key="image"),
        Orientationd(keys=["image"], axcodes="RAS"),
        # user can also add other random transforms
        # RandAffined(
        #     keys=['image', 'label'],
        #     mode=('bilinear', 'nearest'),
        #     prob=1.0, spatial_size=(96, 96, 96),
        #     rotate_range=(0, 0, np.pi/15),
        #     scale_range=(0.1, 0.1, 0.1)),
    ]
)


val_transforms = Compose(
    [
        LoadImaged(keys=["image"], reader="ITKReader", image_only=True), # Douwe: Remove the stupid code below and make sure to have channel_dim as final (kwarks, args)
        SqueezeDimd(keys=["image"], dim=-1), # [1, 18, 256, 256, 1] -> [1, 18, 256, 256] 
        AsChannelLastd(keys=["image"], channel_dim=0), # [1, 18, 256, 256] -> [1, 256, 256, 18] 
        AddChanneld(keys=["image"]), # [1, 18, 256, 256] -> [1, 1, 256, 256, 18] 
    ]
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
<class 'monai.transforms.utility.dictionary.AddChanneld'>: Class `AddChanneld` has been deprecated since version 0.8. It will be removed in version 1.3. please use MetaTensor data type and monai.transforms.EnsureChannelFirstd instead with `channel_dim='no_channel'`.
monai.transforms.utility.dictionary EnsureChannelFirstd.__init__:meta_keys: Argument `meta_keys` has been deprecated since version 0.9. not needed if image is type `MetaTensor`.


In [116]:
from monai.visualize import matshow3d
check_ds = Dataset(data=test_dataset, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
image = (check_data["image"])
print(f"image shape: {image.shape}")

matshow3d(
    volume=check_data["image"],
    fig=None,
    title="input image",
    figsize=(100, 100),
    frame_dim=-1,
    show=True,
    cmap="gray",
)

image shape: torch.Size([1, 1, 256, 256, 18])


(<Figure size 10000x10000 with 1 Axes>,
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan,  0.,  0., ...,  0.,  0., nan],
        [nan,  0.,  0., ...,  0.,  0., nan],
        ...,
        [nan,  0.,  0., ..., nan, nan, nan],
        [nan,  0.,  0., ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]], dtype=float32))

In [ ]:
# Normalize image and check how it influenes it
val_transforms_2 = Compose(
    [
        LoadImaged(keys=["image"], reader="ITKReader", image_only=True),
        SqueezeDimd(keys=["image"], dim=-1),
        AsChannelLastd(keys=["image"], channel_dim=0),
        AddChanneld(keys=["image"]),
        Orientationd(keys=["image"], axcodes="RAS"),
        # Normalization
    ]
)

# Check for second transform
check_ds = Dataset(data=test_dataset, transform=val_transforms_2)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
image = (check_data["image"])
print(f"image shape: {image.shape}")

matshow3d(
    volume=check_data["image"],
    fig=None,
    title="input image",
    figsize=(100, 100),
    frame_dim=-1,
    show=True,
    cmap="gray",
)